In [2]:
df = spark.sql("SELECT * FROM Silver.sl_order LIMIT 1000")
display(df.head(2))

StatementMeta(, f8b73c8c-8a84-4d0c-84f4-e07de04a65e2, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, e0efa534-a9ae-4f57-8bc1-37e1fc0999b9)

total amount of sales in a hour with a sliding window of 15 mints


In [ ]:


display(
    rQuery.withWatermark('OrderDate', '30 minutes').groupby('PaymentMethod',window(rQuery.OrderDate, '15 minutes', '5 minutes'))
    .agg(count('*'))
    .select(f.col("PaymentMethod"), 
            f.col("window.start").alias('startingTime'),
            f.col("window.end").alias("endTime"),
            f.col("count(1)").alias('paymentTypeCount')

In [20]:
from pyspark.sql.functions import window, count
from pyspark.sql import functions as f


class gold:
    def __init__(self):
        self.base_silver_dir='abfss://cf927cbd-ab77-4aa6-9ebf-d7125aace954@onelake.dfs.fabric.microsoft.com/7432d6ed-1dd7-41f0-8b94-3ce6ca527366/Files'
        self.base_gold_dir='abfss://cf927cbd-ab77-4aa6-9ebf-d7125aace954@onelake.dfs.fabric.microsoft.com/e0225320-0c9a-452a-b60d-a0eb20df52d1/Tables'

    def readSilverStream(self):
        return (
            spark.readStream.table("sl_order")
            )

    def goldAgg(self):
        silverOrder= self.readSilverStream()

        sQuery= silverOrder.withWatermark('OrderDate', '1 hour')\
                           .groupby('PaymentMethod', window(silverOrder.OrderDate, '15 minutes', '5 minutes' ))\
                           .agg(count('*').alias('countPaymentType'))\
                           .select( f.col("PaymentMethod"),
                                    f.col("window.start").alias('startingTime'),
                                    f.col("window.end").alias("endTime"),
                                    f.col("countPaymentType")
                                )
        return sQuery

    def save(self):

        self.goldAgg().writeStream.queryName("gold-ingestion")\
                                            .option("checkpointLocation", f"{self.base_silver_dir}/chekpoint/order")\
                                            .outputMode("complete")\
                                            .start(f"{self.base_gold_dir}/slidingPaymentType")       



        

StatementMeta(, e12e3216-c20a-46cc-b3dc-0143e81f2bac, 24, Finished, Available)

In [21]:
gld= gold()
gld.save()

StatementMeta(, e12e3216-c20a-46cc-b3dc-0143e81f2bac, 25, Finished, Available)

In [22]:
df = spark.sql("SELECT * FROM gold.slidingPaymentType LIMIT 1000")
display(df)

StatementMeta(, e12e3216-c20a-46cc-b3dc-0143e81f2bac, 26, Finished, Available)

SynapseWidget(Synapse.DataFrame, 26e0b766-8afa-4b9d-b89c-b6c7d33e4126)